<a href="https://colab.research.google.com/github/aayush1607/Phishing_Detection/blob/main/Social_Media_based_Phishing_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install arff

  Created wheel for arff: filename=arff-0.9-py3-none-any.whl size=4971 sha256=e675254c5fbb0c9df4c65a8c512a71f33d78441a04a24ead3a0a54d6789f4f30
  Stored in directory: /root/.cache/pip/wheels/fe/81/bd/4ae90e24ba860304e375da219f9205b2586dbee255f3ee70e2
Successfully built arff


In [ ]:
from sklearn.tree import _tree


In [ ]:
def tree_to_json(tree):
    tree_ = tree.tree_
    feature_names = range(30)
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]
    def recurse(node):
        tree_json = dict()
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            tree_json['type'] = 'split'
            threshold = tree_.threshold[node]
            tree_json['threshold'] = "{} <= {}".format(feature_name[node], threshold)
            tree_json['left'] = recurse(tree_.children_left[node])
            tree_json['right'] = recurse(tree_.children_right[node])
        else:
            tree_json['type'] = 'leaf'
            tree_json['value'] = tree_.value[node].tolist()
        return tree_json

    return recurse(0)


In [ ]:
def forest_to_json(forest):
    forest_json = dict()
    forest_json['n_features'] = forest.n_features_
    forest_json['n_classes'] = forest.n_classes_
    forest_json['classes'] = forest.classes_.tolist()
    forest_json['n_outputs'] = forest.n_outputs_
    forest_json['n_estimators'] = forest.n_estimators
    forest_json['estimators'] = [tree_to_json(estimator) for estimator in forest.estimators_]
    return forest_json


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
import numpy as np
import json

In [ ]:
import arff
import numpy as np
import json
from sklearn.model_selection import train_test_split, KFold

In [ ]:
dataset = arff.load(open('dataset.arff', 'r'))
barray = []
for row in arff.load('dataset.arff'):
  barray.append(list(row))

data = np.array(barray)
data

array([['-1', '1', '1', ..., '1', '-1', '-1'],
       ['1', '1', '1', ..., '1', '1', '-1'],
       ['1', '0', '1', ..., '0', '-1', '-1'],
       ...,
       ['1', '-1', '1', ..., '0', '1', '-1'],
       ['-1', '-1', '1', ..., '1', '1', '-1'],
       ['-1', '-1', '1', ..., '1', '-1', '-1']], dtype='<U2')

In [ ]:
print('The dataset has {0} datapoints with {1} features'.format(data.shape[0], data.shape[1]-1))

The dataset has 11055 datapoints with 30 features


In [ ]:
data = data[:, [0, 1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13, 14, 15, 16, 22, 30]]

In [ ]:
X, y = data[:, :-1], data[:, -1]
y.reshape(y.shape[0])
print('Before spliting')
print('X:{0}, y:{1}'.format(X.shape, y.shape))
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
print('After spliting')
print('X_train:{0}, y_train:{1}, X_test:{2}, y_test:{3}'.format(X_train.shape, y_train.shape, X_test.shape, y_test.shape))

Before spliting
X:(11055, 17), y:(11055,)
After spliting
X_train:(7738, 17), y_train:(7738,), X_test:(3317, 17), y_test:(3317,)


In [ ]:
np.save('X_train.npy', X_train)
np.save('X_test.npy', X_test)
np.save('y_train.npy', y_train)
np.save('y_test.npy', y_test)

In [ ]:
test_data = dict()
test_data['X_test'] = X_test.tolist()
test_data['y_test'] = y_test.tolist()
with open('testdata.json', 'w') as tdfile:
    json.dump(test_data, tdfile)
    print('Test Data written to testdata.json')

Test Data written to testdata.json


In [ ]:
X_train = np.load('X_train.npy')
y_train = np.load('y_train.npy')
print('X_train:{0}, y_train:{1}'.format(X_train.shape, y_train.shape))

X_train:(7738, 17), y_train:(7738,)


In [ ]:
clf = RandomForestClassifier()
print('Cross Validation Score: {0}'.format(np.mean(cross_val_score(clf, X_train, y_train, cv=10))))

Cross Validation Score: 0.9467549832693187


In [ ]:
clf.fit(X_train, y_train)

RandomForestClassifier()

In [ ]:
X_test = np.load('X_test.npy')
y_test = np.load('y_test.npy')

In [ ]:
pred = clf.predict(X_test)
print('Accuracy: {}'.format(accuracy_score(y_test, pred)))

Accuracy: 0.9469400060295448


In [ ]:
json.dump(forest_to_json(clf), open('classifier.json', 'w'))

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `n_features_` was deprecated in version 1.0 and will be removed in 1.2. Use `n_features_in_` instead.
  warnings.warn(msg, category=FutureWarning)
